In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [ ]:
from datasets import load_dataset
dataset = load_dataset("RCC-MSU/collection3")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

collection3.py:   0%|          | 0.00/3.68k [00:00<?, ?B/s]

The repository for RCC-MSU/collection3 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/RCC-MSU/collection3.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


train.txt.gz:   0%|          | 0.00/606k [00:00<?, ?B/s]

valid.txt.gz:   0%|          | 0.00/143k [00:00<?, ?B/s]

test.txt.gz:   0%|          | 0.00/130k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9301 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2153 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1922 [00:00<?, ? examples/s]

In [ ]:
# Étiquettes pour les tags (par exemple : {0: "O", 1: "B-LOC", 2: "I-LOC"})
label_list = dataset["train"].features["ner_tags"].feature.names

In [ ]:
# Conversion des indices en étiquettes lisibles
example = dataset["train"][0]
tokens = example["tokens"]
labels = [label_list[tag] for tag in example["ner_tags"]]
print("Tokens:", tokens)
print("NER Tags:", labels)


Tokens: ['Дополнение', ':', 'Д', '.', 'Медведев', 'присвоил', 'звания', 'сотрудников', 'полиции', 'и', 'переназначил', '14', 'руководителей', 'УВД', ',', 'ГУВД', 'и', 'МВД', 'по', 'субъектам', 'РФ', '.']
NER Tags: ['O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'B-ORG', 'O', 'B-ORG', 'O', 'O', 'B-LOC', 'O']


In [ ]:
from transformers import AutoTokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
# Exemple de tokenisation
tokenized_input = tokenizer(tokens, is_split_into_words=True, truncation=True, padding=True)
print(tokenized_input)

{'input_ids': [0, 2718, 152645, 152, 1729, 6, 5, 189322, 440, 66176, 3873, 210, 26464, 102445, 180295, 35, 1741, 212, 70582, 547, 616, 191040, 447, 120617, 6, 4, 6, 47871, 120617, 35, 195817, 129, 60985, 1258, 9403, 6, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, padding=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Ignorer les sous-tokens
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Appliquer la fonction
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/9301 [00:00<?, ? examples/s]

Map:   0%|          | 0/2153 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer


In [ ]:
# Charger le modèle xlm-roberta-base
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_list))

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Entraîneur
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

<ipython-input-16-8ba519331305>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Entraîner
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aboubacardiallo050 (aboubacardiallo050-eni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",  # Ajoute un padding pour rendre les séquences de longueur égale
        is_split_into_words=True,
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:  # Pour les sous-tokens ou le padding
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Premier sous-token du mot
                label_ids.append(label[word_idx])
            else:  # Sous-token du même mot
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# Tokenisation et alignement des labels sur l'ensemble du dataset
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["id", "tokens", "ner_tags"],  # Supprimer les colonnes inutiles après traitement
)

Map:   0%|          | 0/9301 [00:00<?, ? examples/s]

Map:   0%|          | 0/2153 [00:00<?, ? examples/s]

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

In [ ]:
print("Input IDs Length:", len(tokenized_dataset["train"][0]["input_ids"]))
print("Labels Length:", len(tokenized_dataset["train"][0]["labels"]))

Input IDs Length: 512
Labels Length: 512


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer


In [ ]:
# Charger le modèle xlm-roberta-base
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_list))


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)
trainer.train()

<ipython-input-24-bba9cda421ee>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.144400,0.033949
2,0.032600,0.022974
3,0.021900,0.022839


TrainOutput(global_step=1746, training_loss=0.05940112980234118, metrics={'train_runtime': 2972.396, 'train_samples_per_second': 9.387, 'train_steps_per_second': 0.587, 'total_flos': 7291292988976128.0, 'train_loss': 0.05940112980234118, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()


In [ ]:
# Afficher les résultats
print(results)


{'eval_loss': 0.022839415818452835, 'eval_runtime': 59.6227, 'eval_samples_per_second': 36.11, 'eval_steps_per_second': 2.264, 'epoch': 3.0}


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
# Obtenez les prédictions
predictions, labels, _ = trainer.predict(tokenized_dataset["validation"])


In [ ]:
# Convertir les logits en prédictions de classes
predicted_classes = np.argmax(predictions, axis=2)

In [ ]:
# Alignement des prédictions avec les étiquettes originales
true_labels = [
    [label for label, pred in zip(label_seq, pred_seq) if label != -100]
    for label_seq, pred_seq in zip(labels, predicted_classes)
]

In [ ]:
predicted_labels = [
    [pred for label, pred in zip(label_seq, pred_seq) if label != -100]
    for label_seq, pred_seq in zip(labels, predicted_classes)
]

In [ ]:
# Aplatir les séquences pour calculer les métriques
flat_true_labels = [label for seq in true_labels for label in seq]
flat_predicted_labels = [label for seq in predicted_labels for label in seq]

print(tokenized_dataset["train"].column_names)


['input_ids', 'attention_mask', 'labels']


In [ ]:
label_names = [
    "O",        # Outside of a named entity
    "B-PER",    # Beginning of a person's name
    "I-PER",    # Inside a person's name
    "B-ORG",    # Beginning of an organization
    "I-ORG",    # Inside an organization
    "B-LOC",    # Beginning of a location
    "I-LOC"     # Inside a location
]

In [ ]:
print(tokenized_dataset["train"][0])  # Premier échantillon

{'input_ids': [0, 2718, 152645, 152, 1729, 6, 5, 189322, 440, 66176, 3873, 210, 26464, 102445, 180295, 35, 1741, 212, 70582, 547, 616, 191040, 447, 120617, 6, 4, 6, 47871, 120617, 35, 195817, 129, 60985, 1258, 9403, 6, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(flat_true_labels, flat_predicted_labels, target_names=label_names))


              precision    recall  f1-score   support

           O       1.00      1.00      1.00     36570
       B-PER       1.00      1.00      1.00      1641
       I-PER       1.00      1.00      1.00      1615
       B-ORG       0.96      0.96      0.96      1308
       I-ORG       0.95      0.94      0.95      1199
       B-LOC       0.98      0.99      0.99      1168
       I-LOC       0.96      0.98      0.97       355

    accuracy                           0.99     43856
   macro avg       0.98      0.98      0.98     43856
weighted avg       0.99      0.99      0.99     43856

